In [1]:
#load librairies

import numpy as np
import pandas as pd

from keras.layers import Dense, Embedding, LSTM
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences


from sklearn.model_selection import train_test_split
import re

In [2]:
from google.colab import files
uploaded = files.upload()

Saving Twitter Sentiments.csv to Twitter Sentiments.csv


In [3]:
#load data
df = pd.read_csv("Twitter Sentiments.csv")

In [13]:
print(df.columns)
print(df)

Index(['id', 'label', 'tweet'], dtype='object')
          id  label                                              tweet
0          1      0   @user when a father is dysfunctional and is s...
1          2      0  @user @user thanks for #lyft credit i can't us...
2          3      0                                bihday your majesty
3          4      0  #model   i love u take with u all the time in ...
4          5      0             factsguide: society now    #motivation
...      ...    ...                                                ...
31957  31958      0  ate @user isz that youuu?ðððððð...
31958  31959      0    to see nina turner on the airwaves trying to...
31959  31960      0  listening to sad songs on a monday morning otw...
31960  31961      1  @user #sikh #temple vandalised in in #calgary,...
31961  31962      0                   thank you @user for you follow  

[31962 rows x 3 columns]


In [14]:
#remove id column
new_df = df[["label", "tweet"]]
new_df

,label,tweet
0,0,@user when a father is dysfunctional and is s...
1,0,@user @user thanks for #lyft credit i can't us...
2,0,bihday your majesty
3,0,#model i love u take with u all the time in ...
4,0,factsguide: society now #motivation
...,...,...
31957,0,ate @user isz that youuu?ðððððð...
31958,0,to see nina turner on the airwaves trying to...
31959,0,listening to sad songs on a monday morning otw...
31960,1,"@user #sikh #temple vandalised in in #calgary,..."


In [23]:
#preprocessing the tweets
new_df["tweet"] = new_df["tweet"].str.lower()

function = lambda x:re.sub('[^\w\s]','', x)
new_df["tweet"] = new_df["tweet"].apply(function)

print(new_df)

       label                                              tweet
0          0   user when a father is dysfunctional and is so...
1          0  user user thanks for lyft credit i cant use ca...
2          0                                bihday your majesty
3          0  model   i love u take with u all the time in u...
4          0               factsguide society now    motivation
...      ...                                                ...
31957      0                ate user isz that youuuðððððððððâï 
31958      0    to see nina turner on the airwaves trying to...
31959      0  listening to sad songs on a monday morning otw...
31960      1  user sikh temple vandalised in in calgary wso ...
31961      0                    thank you user for you follow  

[31962 rows x 2 columns]


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [24]:
#word embedding
tokenizer = Tokenizer(num_words=1500, split=' ')
tokenizer.fit_on_texts(new_df['tweet'].values)

#Cette fonction transforme une liste (de longueur num_samples) de séquences (listes d'entiers)
# en un tableau Numpy 2D de forme (num_samples, num_timesteps). num_timesteps est soit 
#l'argument maxlen s'il est fourni, soit la longueur de la plus longue séquence de la liste
X = tokenizer.texts_to_sequences(new_df["tweet"])
#print(tokenizer.sequence_to_text(X[0]))
X = pad_sequences(X) 


In [25]:
from tensorflow import keras

#LSTM model
embed_dim = 128
lstm_out = 196

model = keras.Sequential()
model.add(Embedding(1500, embed_dim, input_length = 31))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation = "softmax"))

model.compile(loss = "binary_crossentropy", optimizer='adam', metrics = ['accuracy'])
model.summary()
          

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 31, 128)           192000    
                                                                 
 lstm_2 (LSTM)               (None, 196)               254800    
                                                                 
 dense_2 (Dense)             (None, 1)                 197       
                                                                 
Total params: 446,997
Trainable params: 446,997
Non-trainable params: 0
_________________________________________________________________


In [26]:
#encode label
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(new_df["label"])
print("ft",y)


ft [0 0 0 ... 0 1 0]


In [27]:
#divide in train-test split
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.15, random_state = 42)

y_train = np.asarray(y_train).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [28]:
#train the model
model.fit(x_train, y_train, validation_data = (x_test, y_test), epochs = 10, batch_size = 32)

Epoch 1/10
849/849 [==============================] - 157s 182ms/step - loss: 0.1767 - accuracy: 0.0705 - val_loss: 0.1447 - val_accuracy: 0.0682
Epoch 2/10
849/849 [==============================] - 154s 181ms/step - loss: 0.1255 - accuracy: 0.0705 - val_loss: 0.1438 - val_accuracy: 0.0682
Epoch 3/10
849/849 [==============================] - 153s 180ms/step - loss: 0.1089 - accuracy: 0.0705 - val_loss: 0.1406 - val_accuracy: 0.0682
Epoch 4/10
849/849 [==============================] - 154s 181ms/step - loss: 0.0950 - accuracy: 0.0705 - val_loss: 0.1557 - val_accuracy: 0.0682
Epoch 5/10
849/849 [==============================] - 152s 179ms/step - loss: 0.0830 - accuracy: 0.0705 - val_loss: 0.1652 - val_accuracy: 0.0682
Epoch 6/10
849/849 [==============================] - 154s 182ms/step - loss: 0.0730 - accuracy: 0.0705 - val_loss: 0.1851 - val_accuracy: 0.0682
Epoch 7/10
849/849 [==============================] - 154s 182ms/step - loss: 0.0623 - accuracy: 0.0705 - val_loss: 0.1908 -

In [29]:
model.evaluate(x_test,y_test)



150/150 [==============================] - 4s 25ms/step - loss: 0.2584 - accuracy: 0.0682


[0.2583785653114319, 0.06819603592157364]

In [30]:
predict_x=model.predict(x_test[1:4]) 
classes_x=np.argmax(predict_x,axis=1)

text = tokenizer.sequences_to_texts(x_test[1:4])

# for i in text :
#   print(i, end='\n')

print("dataset", text)

print("Prediction: ",classes_x)

print("Actual: \n",y_test[1:4])

1/1 [==============================] - 0s 330ms/step
dataset ['in 3 days i will be meeting my on our first lifestyle vacation', 'the using this tragedy as a way of not in my name you orlando', 'if this doesnt friday']
Prediction:  [0 0 0]
Actual: 
 [[0.]
 [0.]
 [0.]]
